# Imports

In [ ]:
from math import sqrt
import sys
import pandas as pd
import os
import numpy as np
import joblib

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..','..','..')))

from src.utils.model import load_models_auto, compare_models_per_station

#MODEL = "qrf"
#N = 20  # Number of top features to display


In [ ]:
SEED = 42
USE_AUTO_SCAN = True
NUMBER_OF_WEEK = 4
INPUT_DIR = "../../../../data/input/"
MODEL_DIR = "../../../../models/exploration/"
ALPHA = 0.1

# columns to drop : target at different horizon, station_code, and features removed from Feature Selection
TO_DROP = ["water_flow_week1", "station_code", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

# Loading of the data

In [ ]:
# load the dataset
ds_train = pd.read_csv(f"{INPUT_DIR}ds_train.csv")
ds_test_spatio_temporal = pd.read_csv(f"{INPUT_DIR}ds_test_spatio_temporal.csv")
ds_test_temporal = pd.read_csv(f"{INPUT_DIR}ds_test_temporal.csv")

ds_train["ObsDate"] = pd.to_datetime(ds_train["ObsDate"])
ds_test_spatio_temporal["ObsDate"] = pd.to_datetime(ds_test_spatio_temporal["ObsDate"])
ds_test_temporal["ObsDate"] = pd.to_datetime(ds_test_temporal["ObsDate"])

ds_train = ds_train.set_index("ObsDate")
ds_test_spatio_temporal = ds_test_spatio_temporal.set_index("ObsDate")
ds_test_temporal = ds_test_temporal.set_index("ObsDate")

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [ ]:
X_train = ds_train.drop(columns=TO_DROP)
y_train = {}
y_train[0] = ds_train["water_flow_week1"]
for i in range(1, NUMBER_OF_WEEK):
    y_train[i] = ds_train[f"water_flow_week{i+1}"]

X_test_spatio_temporal = ds_test_spatio_temporal.drop(columns=TO_DROP)
y_test_spatio_temporal = {}
for i in range(0, NUMBER_OF_WEEK):
    y_test_spatio_temporal[i] = ds_test_spatio_temporal[f"water_flow_week{i+1}"]

X_test_temporal = ds_test_temporal.drop(columns=TO_DROP)
y_test_temporal = {}
for i in range(0, NUMBER_OF_WEEK):
    y_test_temporal[i] = ds_test_temporal[f"water_flow_week{i+1}"]

mapie_enbpi = {}
mapie = {}
qrf = {}
mapie_aci = {}



# Loading of the model to analyse

Models must be trained on 04 Modelisation, exemples are 'qrf' and 'mapie'.

In [ ]:
# Load models based on conditions
mapie = []
if USE_AUTO_SCAN:
    mapie = load_models_auto("mapie_quantile", MODEL_DIR)
else:
    mapie.append(joblib.load("../../models/mapie_quantile_2025-01-17_15-15-04_week0.pkl"))
    mapie.append(joblib.load("../../models/mapie_quantile_2025-01-17_15-15-11_week1.pkl"))
    mapie.append(joblib.load("../../models/mapie_quantile_2025-01-17_15-15-17_week2.pkl"))
    mapie.append(joblib.load("../../models/mapie_quantile_2025-01-17_15-15-17_week3.pkl"))

qrf = []
if USE_AUTO_SCAN:
    qrf = load_models_auto("qrf_quantile", MODEL_DIR)
else:
    qrf.append(joblib.load("../../models/qrf_quantile_2025-01-17_15-15-04_week0.pkl"))
    qrf.append(joblib.load("../../models/qrf_quantile_2025-01-17_15-15-11_week1.pkl"))
    qrf.append(joblib.load("../../models/qrf_quantile_2025-01-17_15-15-17_week2.pkl"))
    qrf.append(joblib.load("../../models/qrf_quantile_2025-01-17_15-15-17_week3.pkl"))

ebm_ensemble = []
if USE_AUTO_SCAN:
    ebm_ensemble = load_models_auto("ebm_ensemble", MODEL_DIR)
else:
    ebm_ensemble.append(joblib.load("../../models/EBM_ensemble_2025-01-17_15-15-04_week0.pkl"))
    ebm_ensemble.append(joblib.load("../../models/EBM_ensemble_2025-01-17_15-15-11_week1.pkl"))
    ebm_ensemble.append(joblib.load("../../models/EBM_ensemble_2025-01-17_15-15-17_week2.pkl"))
    ebm_ensemble.append(joblib.load("../../models/EBM_ensemble_2025-01-17_15-15-17_week3.pkl"))


# Performance computation

In the following cells we compute performance on test/train set to check for overfiting

In [ ]:
y_test_stations = ds_test_temporal["station_code"].values
y_train_stations = ds_train["station_code"].values

for i in range(NUMBER_OF_WEEK):

    print(f"============================== WEEK {i} temporal ===================================")
    baseline_day_before = ds_test_temporal["water_flow_lag_1w"]
    y_pred_mapie, y_pis_mapie = mapie[i].predict(X_test_temporal)
    y_pred_qrf = qrf[i].predict(X_test_temporal, quantiles="mean", aggregate_leaves_first=False)
    y_pis_qrf = qrf[i].predict(X_test_temporal, quantiles=[ALPHA/2, 1-ALPHA/2])

    predictions = [
        {"model": "LGBM+MAPIE", "prediction": y_pred_mapie, "dataset":"test", "stations": y_train_stations, "prediction_interval": y_pis_mapie},
        {"model": "Week before", "prediction": baseline_day_before, "dataset":"test", "stations": y_train_stations, "prediction_interval": None},
        {"model": "QRF", "prediction": y_pred_qrf, "dataset":"test", "stations": y_train_stations, "prediction_interval": y_pis_qrf},
    ]


    compare_models_per_station(
        y_test_temporal[i].values,
        predictions,
        y_test_stations,
        column_to_display="log_likelihood" ,
        title = f"WEEK {i}")



In [ ]:
y_test_stations = ds_test_spatio_temporal["station_code"].values

for i in range(NUMBER_OF_WEEK):

    print(f"============================== WEEK {i} temporal ===================================")
    baseline_day_before = ds_test_spatio_temporal["water_flow_lag_1w"]
    y_pred_mapie, y_pis_mapie = mapie[i].predict(X_test_spatio_temporal)
    y_pred_qrf = qrf[i].predict(X_test_spatio_temporal, quantiles="mean", aggregate_leaves_first=False)
    y_pis_qrf = qrf[i].predict(X_test_spatio_temporal, quantiles=[ALPHA/2, 1-ALPHA/2])

    predictions = [
        {"model": "LGBM+MAPIE", "prediction": y_pred_mapie, "dataset":"test", "stations": y_test_stations, "prediction_interval": y_pis_mapie},
        {"model": "Week before", "prediction": baseline_day_before, "dataset":"test", "stations": y_test_stations, "prediction_interval": None},
        {"model": "QRF", "prediction": y_pred_qrf, "dataset":"test", "stations": y_test_stations, "prediction_interval": y_pis_qrf},
    ]


    compare_models_per_station(
        y_test_spatio_temporal[i].values,
        predictions,
        y_test_stations,
        column_to_display="log_likelihood" ,
        title = f"WEEK {i}")